In [2]:
import requests
import pandas as pd
import json

In [3]:
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

In [4]:
query = """
[out:json][timeout:120];
area["name"="Rijswijk"]["boundary"="administrative"]->.a;
(
  node(area.a)["amenity"];
  node(area.a)["shop"];
  node(area.a)["tourism"];
  node(area.a)["leisure"];
  node(area.a)["public_transport"];
  node(area.a)["office"];
  way(area.a)["highway"];
);
out center;
"""


In [5]:
print("🔍 Querying Overpass API ...")
response = requests.get(OVERPASS_URL, params={'data': query})
response.raise_for_status()
data = response.json()

print(f"✅ Retrieved {len(data['elements'])} map elements")

🔍 Querying Overpass API ...
✅ Retrieved 8785 map elements


In [6]:
records = []
for el in data['elements']:
    tags = el.get('tags', {})
    name = tags.get('name')
    lat = el.get('lat') or el.get('center', {}).get('lat')
    lon = el.get('lon') or el.get('center', {}).get('lon')

    # Combine all possible categories
    category = (
        tags.get('amenity')
        or tags.get('shop')
        or tags.get('tourism')
        or tags.get('leisure')
        or tags.get('public_transport')
        or tags.get('office')
        or tags.get('highway')
        or 'unknown'
    )

    records.append({
        'name': name or '(unnamed)',
        'category': category,
        'lat': lat,
        'lon': lon,
        'tags': json.dumps(tags, ensure_ascii=False)
    })

# --- Convert to DataFrame ---
df = pd.DataFrame(records)
print(df.head())

# --- Save results ---
df.to_csv("rijswijk_pois.csv", index=False, encoding='utf-8')
print("📁 Saved results to rijswijk_pois.csv")

# Optional: save as GeoJSON for mapping tools
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": [r['lon'], r['lat']]},
            "properties": {"name": r['name'], "category": r['category'], "tags": json.loads(r['tags'])}
        }
        for _, r in df.iterrows() if pd.notna(r['lat']) and pd.notna(r['lon'])
    ]
}

with open("rijswijk_pois.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print("🗺️ Saved GeoJSON file: rijswijk_pois.geojson")

                  name          category        lat       lon  \
0            (unnamed)         recycling  52.038247  4.318688   
1          Broekpolder     stop_position  52.049760  4.345469   
2           McDonald's         fast_food  52.035974  4.316177   
3                 ALDI       supermarket  52.056811  4.337401   
4  K.R.S.V Vredenburch  community_centre  52.050188  4.322008   

                                                tags  
0  {"amenity": "recycling", "location": "undergro...  
1  {"name": "Broekpolder", "network": "Rail Haagl...  
2  {"addr:city": "Rijswijk", "addr:housenumber": ...  
3  {"brand": "ALDI", "brand:wikidata": "Q41171373...  
4  {"addr:city": "Rijswijk", "addr:housenumber": ...  
📁 Saved results to rijswijk_pois.csv
🗺️ Saved GeoJSON file: rijswijk_pois.geojson


In [3]:
df = pd.read_csv('rijswijk_pois.csv')

In [8]:
print(df["category"].unique().tolist())

['recycling', 'stop_position', 'fast_food', 'supermarket', 'community_centre', 'sports_centre', 'parking_entrance', 'fuel', 'parking', 'hotel', 'post_box', 'bicycle_parking', 'bicycle', 'doctors', 'pharmacy', 'viewpoint', 'picnic_table', 'bench', 'car_wash', 'restaurant', 'alcohol', 'atm', 'books', 'pub', 'gift', 'tobacco', 'cafe', 'vacant', 'platform', 'waste_basket', 'information', 'artwork', 'department_store', 'kindergarten', 'electronics', 'seafood', 'place_of_worship', 'bakery', 'hairdresser', 'dry_cleaning', 'outdoor', 'mobile_phone', 'confectionery', 'houseware', 'it', 'company', 'car_repair', 'florist', 'clinic', 'estate_agent', 'interior_decoration', 'childcare', 'convenience', 'hardware', 'dentist', 'notary', 'fitness_centre', 'pet', 'driving_school', 'clothes', 'brothel', 'school', 'chemist', 'music_school', 'butcher', 'wool', 'party', 'newsagent', 'social_facility', 'library', 'museum', 'beauty', 'wine', 'greengrocer', 'ice_cream', 'baby_goods', 'deli', 'toys', 'shoe_repai

In [11]:
df_stop_positions = df[
    (df["name"] != "(unnamed)") & (df["category"].isin(["educational_institution","fire_station", "stop_position", "museum", "toys", "kindergarten", "zoo", "storage_rental", "hotel", "school", "government"]))
]

print(len(df_stop_positions))
df_stop_positions["name"].unique()

157


array(['Broekpolder', "Rijswijk, 't Haantje",
       'Rijswijk, Wethouder Brederodelaan', 'Rijswijk',
       'Rijswijk, Huis te Landelaan', 'Rijswijk, Van Vredenburchweg',
       'Eikelenburg', 'Weidedreef', 'Bogaard stadscentrum', 'Treubstraat',
       'Volmerlaan', 'Handelskade', 'Generaal Spoorlaan',
       'Aletta Jacobsstraat', 'Patentlaan', 'Rijswijk, Station',
       'Admiraal Helfrichsingel', 'Station Rijswijk',
       'PLAZA Premium Grand Winston', 'Van Vredenburchweg', 'Verffabriek',
       'Rijswijk, Generaal Spoorlaan', "'s-Gravenmade",
       'Volmerlaan (Zuid)', "Den Haag, Laan van 's-Gravenmade",
       'Rijswijk, Albert Schweitzerlaan', 'Rijswijk, Aletta Jacobsstraat',
       'Rijswijk, Sir Winston Churchilllaan', 'Herenstraat', 'Hoornbrug',
       'Rijswijk, Esdoornstraat', 'Rijswijk, Doctor H. Colijnlaan',
       'Rijswijk, Hendrik Ravesteijnplein', 'Rijswijk, Paulus Potterlaan',
       'Rijswijk, Klaroenstraat', 'Rijswijk, Admiraal Helfrichsingel',
       'Rijswijk, 